# Decorators

In [4]:
from datetime import date
def now():
    print(date.today())

f = now
f()

2019-12-04


In [5]:
now.__name__

'now'

In [6]:
f.__name__

'now'

现在，假设我们要增强`now()`函数的功能，比如，在函数调用前后自动打印日志，但又不希望修改now()函数的定义，这种在代码运行期间动态增加功能的方式，称之为“装饰器”（Decorator）。

本质上，decorator就是一个`返回函数`的高阶函数。

In [7]:
def log(func):
    def wrapper(*arg,**kw):
        print('call %s():'%func.__name__)
        return func(*arg,**kw)
    return wrapper

In [9]:
# Method 1:

ff = log(now)
ff()

call now():
2019-12-04


In [14]:
# Method 2:

from datetime import date
@log
def now():
    print(date.today())
    
now()

call now():
2019-12-04


把`@log`放到`now()`函数的定义处，相当于执行了语句：
```python
now = log(now)
```

由于`log()`是一个decorator，返回一个函数，所以，原来的`now()`函数仍然存在，只是现在同名的now变量指向了新的函数，于是调用now()将执行新函数，即在log()函数中返回的wrapper()函数。

wrapper()函数的参数定义是`(*args, **kw)`，因此，wrapper()函数可以接受任意参数的调用。在wrapper()函数内，首先打印日志，再紧接着调用原始函数。

### 含有参数的def

In [21]:
def log(text):
    def decorator(func):
        def wrapper(*args, **kw):
            print('%s %s():' % (text, func.__name__))
            return func(*args, **kw)
        return wrapper
    return decorator

In [22]:
from datetime import date

@log('execute')
def now():
    print(date.today())

now()    

execute now():
2019-12-04


In [24]:
now.__name__

# now 执行的其实是 wrapper

'wrapper'

In [16]:
from functools import wraps
from time import time

def timeing(fun):
    @wraps(fun)
    def wrapper(*args, **kw):
        start = time()
        result = fun(*args,**kw)
        end = time()
        print('runing time: {time}'.format(time = end-start))
        return result
    return wrapper 

In [12]:
import math

def power(a, p):
    return a**p

def factorial(a):
    res = 1
    for i in range(1,a+1):
        res *= i
    return res
    

In [11]:
print(power(2,5))

32


In [13]:
print(factorial(6))

720


In [17]:
@timeing
def power(a, p):
    return a**p

@timeing
def factorial(a):
    res = 1
    for i in range(1,a+1):
        res *= i
    return res

In [18]:
print(power(2,5))

runing time: 1.6689300537109375e-06
32


In [19]:
print(factorial(10))

runing time: 2.1457672119140625e-06
3628800


### multiple decorators

In [32]:
from functools import wraps
from time import time

def nameit(fun):
    @wraps(fun)
    def wrapper(*args, **kwargs):
        print('NAME: {name} ...'.format(name= fun.__name__))
        return fun(*args,**kwargs)
    return wrapper

def timeing(fun):
    @wraps(fun)
    def wrapper(*args, **kw):
        print('count {name} running time..'.format(name= fun.__name__))
        start = time()
        result = fun(*args,**kw)
        end = time()
        print('runing time: {time}'.format(time = end-start))
        return result
    return wrapper 

In [33]:
@nameit
@timeing
def foo(name):
    print('hello ' + name)
    
foo('David')

NAME: foo ...
count foo running time..
hello David
runing time: 1.6927719116210938e-05


In [34]:
@timeing
@nameit
def foo(name):
    print('hello ' + name)
    
foo('David')

count foo running time..
NAME: foo ...
hello David
runing time: 3.790855407714844e-05
